# Taxonomic classification and filtering

Note. using the denoised FeatureTable with 200 trunc, POA 4 

## Setup 

In [1]:
wd = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS'
%cd $wd 

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS


In [2]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [ ]:
from biom import load_table
import pandas as pd 
from bs4 import BeautifulSoup
import os
import subprocess
import json
import matplotlib.pyplot as plt

import qiime2 as q2

from qiime2 import (Artifact,
                    Metadata as qmd)

from qiime2.plugins import (cutadapt,
                            demux,
                            feature_table as qft,
                            taxa as q2t,)

from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.feature_table.methods import (merge_seqs, merge, filter_seqs, filter_samples, filter_features, relative_frequency) 
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
from qiime2.plugins.taxa.methods import filter_table, collapse
import qiime2.plugins.metadata.actions as metadata_actions
import qiime2.plugins.taxa.actions as taxa_actions

%matplotlib inline

# Load denoised artifacts
Sven testing (see Appendix) - conditions that worked best: 
1. cutadapt-trim for reverse complement primer with decreased max error tolerance to 5 % 
2. DADA2 **single-end** denoising with truncation at 190 and increased acceptable error rate (4.0) and p-fold abundance (4.0) 


In [25]:
## Load dada table and rep seqs  
# these artifacts were generated by Sven Stoltenberg in testing various denoising parameters 

rep_seq_file = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza'
table_file = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-table.qza'

rep_seqs = q2.Artifact.load(rep_seq_file)
table = q2.Artifact.load(table_file)

In [ ]:
metadata = '/home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv'
md = Metadata.load(metadata)

## Classify 

In [ ]:
%%bash 

qiime feature-classifier classify-sklearn \
    --i-reads /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-classifier /home/lfloerl/public/Data/Databases/QIIME2/UNITE/unite_ver10_99_all_04.04.2024-Q2-2024.5-colinbrinslaw.qza \
    --p-n-jobs 1 \
    --o-classification taxonomy.qza

In [ ]:
!qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv

In [ ]:
Visualization.load('taxonomy.qzv')

In [ ]:
# table loaded above 
taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=md
)
taxa_bar_plots_viz.save('taxa_barplot_raw.qzv')

In [ ]:
Visualization.load('taxa_barplot_raw.qzv')

# Quality Control: Decontam 

decontam considering the concentrations and with default parameters

Note, does not work on a batch level (ie. here per Plate) because we don't have enough controls (5 recommended!)

In [ ]:
%%bash 

qiime quality-control decontam-identify \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-table.qza \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
    --p-method 'combined' \
    --p-freq-concentration-column "Fungal conc. (ng/uL)" \
    --p-prev-control-column "CTRL" \
    --p-prev-control-indicator "NegCtrl" \
    --o-decontam-scores decontam_scores.qza \

# histogram 
qiime quality-control decontam-score-viz \
    --i-decontam-scores decontam_scores.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-table.qza \
    --o-visualization decontam_histogram.qzv

# actually remove the contaminations 
qiime quality-control decontam-remove \
    --i-decontam-scores decontam_scores.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-table.qza \
    --i-rep-seqs /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --o-filtered-table decontam-table.qza \
    --o-filtered-rep-seqs decontam-rep-seqs.qza


In [ ]:
!qiime feature-table summarize --i-table decontam-table.qza --o-visualization decontam-table.qzv

In [ ]:
Visualization.load('decontam_histogram.qzv')

In [ ]:
Visualization.load('decontam-table.qzv')

## Filter out non-target seqs 

we remove classes of fungi that form a fruiting body and whos DNA can be considered a contamination (from e.g. spores), remove ASVs not classified to at least phylum level, and with a feature count of at least 10.

### Filter the fruiting body forming mushrooms 
> Here we use the whole UNITE database that was combined with the Fungal Traits from Sven Stoltenberg and Anton Lavrinienko to ultiamtely filter all macrofungi from our data. We sort by furiting body type - the list of which fungi form bodies was compiled with the ETH Fungarium. 

In [ ]:
# read in the whole list of taxa and lifestyles 
fungi_df = pd.read_csv('/home/lfloerl/microterroir/Microbiome/ITS/UNITE-FungalTraits.csv')

# these lifestyles are macrofungi! 
fruting_body_forming = ['agaricoid',
 'apothecium_(hymenium on surface)',
 'clathroid',
 'clavarioid',
 'corticioid',
 'cyphelloid',
 'gasteroid',
 'gasteroid-hypogeous',
 'hysterothecium_(opening_slith-like)',
 'mazaedium_(pushpin-like)',
 'phalloid',
 'polyporoid',
 'tremelloid']

# filter accordingly 
matching_taxons = fungi_df[fungi_df['Fruitbody_type_template'].isin(fruting_body_forming)]['Taxon'].tolist()

In [ ]:
# number of taxons from the whole UNITE database that forms furiting bodies 
len(matching_taxons)

In [ ]:
# we export the taxonomy 
!qiime tools export --input-path taxonomy.qza --output-path taxonomy 

In [ ]:
# and import as df to get all Taxons we actually identified
taxonomy_df = pd.read_csv('taxonomy/taxonomy.tsv', sep='\t')

In [ ]:
filtered_matching_taxons = [taxon for taxon in matching_taxons if taxon in taxonomy_df['Taxon'].tolist()]

# actual taxons we need to filter out of our FeatureTable 
len(filtered_matching_taxons)

In [ ]:
# Set to hold unique genera
unique_entries = set()

# List to hold entries without 'f__'
entries_without_f = []

# Process each string in the list
for entry in filtered_matching_taxons:
    # Find the start and end of the substring to extract
    start_index = entry.find('f__')
    
    if start_index != -1:
        start_index += len('f__')
        end_index = entry.find(';g__')
        
        if end_index != -1 and end_index > start_index:
            # Extract the substring between 'f__' and ';g'
            substring = entry[start_index:end_index]
            # Add the substring to the set of unique entries
            unique_entries.add(substring)
    else:
        # If 'f__' is not found, add the entry to the list of entries without 'f__'
        entries_without_f.append(entry)

# Convert the set of unique entries to a list
unique_list = list(unique_entries)

# Output the results
print("Unique genera:")
print(unique_list)
print('\n Lenght: ')
print(len(unique_list))
print("\n Just checking - any entries without?")
print(entries_without_f)

In [ ]:
# we use these to filter our FeatureTable 
exclude_fungi_list = ','.join(unique_list)

#with open('exclude_fungi_list.txt', 'w') as f:
#    f.write(exclude_fungi_list)

In [ ]:
exclude_fungi_list

### Filter! 

In [ ]:
# load artifacts
taxonomy = q2.Artifact.load('taxonomy.qza')
decontam_table = q2.Artifact.load('decontam-table.qza')

somehow the API filtering is not reliable! 

In [ ]:
# # filter out fruiting body forming mushrooms and ASVs classified to at least the phylum level!

# taxa_filtered_table, = q2t.actions.filter_table(
#                                 table=decontam_table, 
#                                 taxonomy=taxonomy,
#                                 exclude=exclude_fungi_list)

# phylum_filtered_table, = q2t.actions.filter_table(
#                                 table=taxa_filtered_table,
#                                 taxonomy=taxonomy,
#                                 include='p__')

# # filter samples with less than 10 different features
# feature_filtered_table, = filter_samples(
#                                 table=phylum_filtered_table,
#                                 min_features=10,
#                                 metadata=md)

# # filter the table to only contain samples listed in the metadata (remove ctrls and PNA test)
# md_filtered_table, = filter_samples(table=feature_filtered_table,
#                                 metadata=md_fermented)

# taxa_filtered_table.save('non_target_filtered_table.qza')
# phylum_filtered_table.save('phylum_filtered_table.qza')
# feature_filtered_table.save('feature_filtered_table.qza')
# md_filtered_table.save('taxa_filtered_table.qza')

In [ ]:
%%bash 

filter='Peniophoraceae,Auriscalpiaceae,Botryobasidiaceae,Entolomataceae,Strophariaceae,Corticiaceae,Bolbitiaceae,Irpicaceae,Hymenogastraceae,Sebacinaceae,Schizoporaceae,Typhulaceae,Psathyrellaceae,Hydnodontaceae,Polyporales_fam_Incertae_sedis,Mycenaceae,Stereaceae,Marasmiaceae,Geoglossaceae,Oliveoniaceae,Meruliaceae,Auriculariaceae,Geastraceae,Steccherinaceae,Pleurotaceae,Agaricales_fam_Incertae_sedis,Hymenochaetaceae,Physalacriaceae,Rickenellaceae,Amylocorticiaceae,Fomitopsidaceae,Thelephoraceae,Clavariaceae,Agaricaceae,Atheliaceae,Suillaceae,Phanerochaetaceae,Phallaceae,Ceratobasidiaceae,Inocybaceae,Pluteaceae'

# mushrooms
qiime taxa filter-table \
  --i-table decontam-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-exclude $filter \
  --o-filtered-table non_target_filtered_table.qza

# filter fungi
qiime taxa filter-table \
  --i-table non_target_filtered_table.qza \
  --i-taxonomy taxonomy.qza \
  --p-include 'k__Fung' \
  --o-filtered-table fungi_filtered_table.qza

# classified to feature level
qiime taxa filter-table \
  --i-table fungi_filtered_table.qza \
  --i-taxonomy taxonomy.qza \
  --p-include 'p__' \
  --o-filtered-table phylum_filtered_table.qza
  
# min features 
qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --p-min-features 10 \
  --o-filtered-table feature_filtered_table.qza
  
qiime feature-table filter-samples \
  --i-table feature_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --o-filtered-table taxa_filtered_table.qza

In [11]:
!qiime feature-table summarize --i-table fungi_filtered_table.qza --o-visualization fungi_filtered_table.qzv
!qiime feature-table summarize --i-table phylum_filtered_table.qza --o-visualization phylum_filtered_table.qzv
!qiime feature-table summarize --i-table feature_filtered_table.qza --o-visualization feature_filtered_table.qzv
!qiime feature-table summarize --i-table taxa_filtered_table.qza --o-visualization taxa_filtered_table.qzv

Saved Visualization to: non_target_filtered_table.qzv
Saved Visualization to: fungi_filtered_table.qzv
Saved Visualization to: phylum_filtered_table.qzv
Saved Visualization to: feature_filtered_table.qzv
Saved Visualization to: taxa_filtered_table.qzv


In [13]:
Visualization.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-table.qzv')

<visualization: Visualization uuid: 41cbbe1a-f5fb-41b2-a502-75224e57f8c9>

In [14]:
Visualization.load('decontam-table.qzv')

<visualization: Visualization uuid: 05b263c3-1a48-4a40-a9f1-5d9d29f3278a>

In [15]:
Visualization.load('fungi_filtered_table.qzv')

<visualization: Visualization uuid: 196d3a1f-6b8d-4fdc-8140-da0d0975b011>

In [16]:
Visualization.load('phylum_filtered_table.qzv')

<visualization: Visualization uuid: ff1f865e-b826-4fe8-b588-4d5e61857e40>

In [17]:
Visualization.load('feature_filtered_table.qzv')

<visualization: Visualization uuid: 573c9f6e-365e-460f-bbf5-218b6af14ef0>

In [18]:
Visualization.load('taxa_filtered_table.qzv')

<visualization: Visualization uuid: 4db29309-92cb-42b1-85ed-5fd49a9a50b9>

### Taxa barplot

In [ ]:
%%bash
qiime taxa barplot \
  --i-table phylum_filtered_table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --o-visualization taxa_barplot.qzv

In [ ]:
Visualization.load('taxa_barplot.qzv')

### Filter Representative Sequences accordingly

In [19]:
from qiime2.plugins.feature_table.methods import filter_seqs
taxa_filtered_table = Artifact.load('taxa_filtered_table.qza')
filtered_rep_seqs, = filter_seqs(data=rep_seqs, table=taxa_filtered_table)
filtered_rep_seqs.save('filtered_rep_seqs.qza')

'filtered_rep_seqs.qza'

In [20]:
!qiime feature-table tabulate-seqs --i-data filtered_rep_seqs.qza --o-visualization filtered_rep_seqs.qzv

Saved Visualization to: filtered_rep_seqs.qzv


In [21]:
Visualization.load('filtered_rep_seqs.qzv')

<visualization: Visualization uuid: 60a2efe1-fbab-4331-bfef-7c7457f5b9e7>

## Taxa Barplot

In [ ]:
taxa_filtered_table = Artifact.load('taxa_filtered_table.qza')
taxonomy = Artifact.load('taxonomy.qza')

In [ ]:
taxa_bar_plots_viz, = taxa_actions.barplot(
    table=taxa_filtered_table,
    taxonomy=taxonomy,
    metadata=md,
)
taxa_bar_plots_viz.save('taxa_barplot.qzv')

In [ ]:
Visualization.load('taxa_barplot.qzv')

# Check how many samples/reads per project

In [49]:
md_df = md.to_dataframe()
samples_df = md_df[md_df['CTRL'] == 'Sample']
project_counts = samples_df['Project'].value_counts()
project_counts

Project
Lavaux                  595
SoilColonialization     115
NOT-USE                 106
Valais                   85
SamplingBenchmarking     84
BotrytizedWine           78
WINE                     38
Name: count, dtype: int64

In [ ]:
%%bash 

#mkdir projects

qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='WINE' AND [CTRL]='Sample'" \
  --o-filtered-table projects/WINE_filtered_table.qza
  
qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='BotrytizedWine' AND [CTRL]='Sample'" \
  --o-filtered-table projects/BotrytizedWine_filtered_table.qza
  
qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='Lavaux' AND [CTRL]='Sample'" \
  --o-filtered-table projects/Lavaux_filtered_table.qza

qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='SoilColonialization' AND [CTRL]='Sample'" \
  --o-filtered-table projects/SoilColonialization_filtered_table.qza

qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='SamplingBenchmarking' AND [CTRL]='Sample'" \
  --o-filtered-table projects/SamplingBenchmarking_filtered_table.qza

qiime feature-table filter-samples \
  --i-table phylum_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_md.tsv \
  --p-where "[Project]='Valais' AND [CTRL]='Sample'" \
  --o-filtered-table projects/Valais_filtered_table.qza

qiime feature-table summarize --i-table projects/WINE_filtered_table.qza --o-visualization projects/WINE_filtered_table.qzv
qiime feature-table summarize --i-table projects/BotrytizedWine_filtered_table.qza --o-visualization projects/BotrytizedWine_filtered_table.qzv
qiime feature-table summarize --i-table projects/Lavaux_filtered_table.qza --o-visualization projects/Lavaux_filtered_table.qzv
qiime feature-table summarize --i-table projects/SoilColonialization_filtered_table.qza --o-visualization projects/SoilColonialization_filtered_table.qzv
qiime feature-table summarize --i-table projects/SamplingBenchmarking_filtered_table.qza --o-visualization projects/SamplingBenchmarking_filtered_table.qzv
qiime feature-table summarize --i-table projects/Valais_filtered_table.qza --o-visualization projects/Valais_filtered_table.qzv

In [40]:
Visualization.load('phylum_filtered_table.qzv')

<visualization: Visualization uuid: ff1f865e-b826-4fe8-b588-4d5e61857e40>

In [41]:
Visualization.load('projects/WINE_filtered_table.qzv')

<visualization: Visualization uuid: 2e842424-f9ea-481b-a3e9-2124abc31dad>

In [42]:
Visualization.load('projects/BotrytizedWine_filtered_table.qzv')

<visualization: Visualization uuid: e03f948f-6894-47ed-807a-04b8b33ed299>

In [13]:
Visualization.load('projects/Lavaux_filtered_table.qzv')

<visualization: Visualization uuid: 957fef2b-bc47-4885-a3ad-dad97c28f053>

In [44]:
Visualization.load('projects/SoilColonialization_filtered_table.qzv')

<visualization: Visualization uuid: acb58bb3-9901-49da-8650-7752496de07a>

In [45]:
Visualization.load('projects/SamplingBenchmarking_filtered_table.qzv')

<visualization: Visualization uuid: e5dfd7ee-65de-4463-862a-05e0c7e89d54>

In [46]:
Visualization.load('projects/Valais_filtered_table.qzv')

<visualization: Visualization uuid: d68d2c1f-d733-48ad-a37c-95df0a38ef31>

# Check how many samples/reads per sample type in Lavaux

In [5]:
#!mkdir lavaux

In [6]:
md = q2.Metadata.load('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv')

md_df = md.to_dataframe()
sample_type_counts = md_df['sample type'].value_counts()
sample_type_counts

sample type
must        356
Post-AF      56
soil         55
leaf         53
bark         33
Post-MLF     22
MV-must      20
Name: count, dtype: int64

In [7]:
table = Artifact.load('projects/Lavaux_filtered_table.qza')

# berry samples 
must_filtered_table, = filter_samples(table=table, metadata=md, where="[sample type]='must'")
must_filtered_table.save('lavaux/must_filtered_table.qza')
!qiime feature-table summarize --i-table lavaux/must_filtered_table.qza --o-visualization lavaux/must_filtered_table.qzv

# microvinification samples 
mv_filtered_table, = filter_samples(table=table, metadata=md, 
                                    where="[sample type]='Post-AF' OR [sample type]='Post-MLF' OR [sample type]='MV-must'")
mv_filtered_table.save('lavaux/mv_filtered_table.qza')
!qiime feature-table summarize --i-table lavaux/mv_filtered_table.qza --o-visualization lavaux/mv_filtered_table.qzv

# soil samples 
soil_filtered_table, = filter_samples(table=table, metadata=md, where="[sample type]='soil'")
soil_filtered_table.save('lavaux/soil_filtered_table.qza')
!qiime feature-table summarize --i-table lavaux/soil_filtered_table.qza --o-visualization lavaux/soil_filtered_table.qzv

# leaf samples 
leaf_filtered_table, = filter_samples(table=table, metadata=md, where="[sample type]='leaf'")
leaf_filtered_table.save('lavaux/leaf_filtered_table.qza')
!qiime feature-table summarize --i-table lavaux/leaf_filtered_table.qza --o-visualization lavaux/leaf_filtered_table.qzv

# bark samples 
bark_filtered_table, = filter_samples(table=table, metadata=md, where="[sample type]='bark'")
bark_filtered_table.save('lavaux/bark_filtered_table.qza')
!qiime feature-table summarize --i-table lavaux/bark_filtered_table.qza --o-visualization lavaux/bark_filtered_table.qzv

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan na

Saved Visualization to: lavaux/must_filtered_table.qzv


/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan na

Saved Visualization to: lavaux/mv_filtered_table.qzv


/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan na

Saved Visualization to: lavaux/soil_filtered_table.qzv


/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan na

Saved Visualization to: lavaux/leaf_filtered_table.qzv


/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/qiime2/metadata/metadata.py:610: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan na

Saved Visualization to: lavaux/bark_filtered_table.qzv


In [8]:
Visualization.load('lavaux/must_filtered_table.qzv')

<visualization: Visualization uuid: 3a3480e1-3ded-4aeb-9b61-6ce3afc75f11>

In [9]:
Visualization.load('lavaux/mv_filtered_table.qzv')

<visualization: Visualization uuid: 235a27a3-b41a-466d-8ca9-a7d3b97f6c0a>

In [6]:
Visualization.load('lavaux/soil_filtered_table.qzv')

<visualization: Visualization uuid: 90e5e561-bacc-4db9-885c-7fad52b1a9dc>

In [5]:
Visualization.load('lavaux/leaf_filtered_table.qzv')

<visualization: Visualization uuid: 10a84e25-2d87-462f-b84a-42cadba7024e>

In [4]:
Visualization.load('lavaux/bark_filtered_table.qzv')

<visualization: Visualization uuid: a5cfb3fa-aecf-452e-b216-3462f72723d4>